In [12]:
%matplotlib inline
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms, utils, models
from torch.autograd import Variable

from skimage import io, transform
import numpy as np
import csv
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')


# from multiprocessing import set_start_method
# try:
#     set_start_method('spawn')
# except RuntimeError:
#     pass

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1,2"
# os.environ["CUDA_VISIBLE_DEVICES"]=""
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
# torch.cuda.set_device(0) 
torch.backends.cudnn.enabled = True
print(torch.cuda.device_count())
gpus = [1,2]

1


In [13]:
def expand_bbox(left, right, top, bottom, img_width, img_height):
    width = right-left
    height = bottom-top
    ratio = 0.15
    new_left = np.clip(left-ratio*width,0,img_width)
    new_right = np.clip(right+ratio*width,0,img_width)
    new_top = np.clip(top-ratio*height,0,img_height)
    new_bottom = np.clip(bottom+ratio*height,0,img_height)
    return [int(new_left), int(new_top), int(new_right), int(new_bottom)]

def display_pose(img, pose):
    pose  = pose.data.cpu().numpy().reshape([-1,2])
    img = img.cpu().numpy().transpose(1,2,0)
    img_width, img_height,_ = img.shape
    ax = plt.gca()
    plt.imshow(img)
    for idx in range(16):
        plt.plot(pose[idx,0], pose[idx,1], marker='o', color='yellow')
    xmin = np.min(pose[:,0])
    ymin = np.min(pose[:,1])
    xmax = np.max(pose[:,0])
    ymax = np.max(pose[:,1])
    bndbox = np.array(expand_bbox(xmin, xmax, ymin, ymax, img_width, img_height))
    coords = (bndbox[0], bndbox[1]), bndbox[2]-bndbox[0]+1, bndbox[3]-bndbox[1]+1
    ax.add_patch(plt.Rectangle(*coords, fill=False, edgecolor='yellow', linewidth=2))

In [14]:
%%sh 
jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [15]:
class Rescale(object):

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image_, pose_ = sample['image'], sample['pose']

        h, w = image_.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        image = transform.resize(image_, (new_h, new_w))
        pose = (pose_.reshape([-1,2])/np.array([w,h])*np.array([new_w,new_h])).flatten()
        return {'image': image, 'pose': pose}

class ToTensor(object):

    def __call__(self, sample):
        image, pose = sample['image'], sample['pose']
 
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        mean=np.array([0.485, 0.456, 0.406])
        std=np.array([0.229, 0.224, 0.225])
        image = torch.from_numpy(((image-mean)/std).transpose((2, 0, 1))).float()
        pose = torch.from_numpy(pose).float()
        
        return {'image': image,
                'pose': pose}

class PoseDataset(Dataset):

    def __init__(self, csv_file, transform):
        
        with open(csv_file) as f:
            self.f_csv = list(csv.reader(f, delimiter='\t'))
        self.transform = transform

    def __len__(self):
        return len(self.f_csv)

    def __getitem__(self, idx):
        ROOT_DIR = "/home/yuliang/code/deeppose_tf/datasets/mpii"
        line = self.f_csv[idx][0].split(",")
        img_path = os.path.join(ROOT_DIR,'images',line[0])
        image = io.imread(img_path)
        
        height, width = image.shape[0], image.shape[1]
        pose = np.array([float(item) for item in line[1:]]).reshape([-1,2])
        
        xmin = np.min(pose[:,0])
        ymin = np.min(pose[:,1])
        xmax = np.max(pose[:,0])
        ymax = np.max(pose[:,1])
        
        box = expand_bbox(xmin, xmax, ymin, ymax, width, height)
        image = image[box[1]:box[3],box[0]:box[2],:]
        pose = (pose-np.array([box[0],box[1]])).flatten()
        
        sample = {'image': image, 'pose':pose}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [16]:
ROOT_DIR = "/home/yuliang/code/deeppose_tf/datasets/mpii"
BATCH_SIZE = 64
train_dataset = PoseDataset(csv_file=os.path.join(ROOT_DIR,'train_joints.csv'),
                                  transform=transforms.Compose([
                                               Rescale((227,227)),
                                               ToTensor()
                                           ]))
# train_dataloader = DataLoader(train_dataset, batch_size=256,
#                         shuffle=False, num_workers = 10)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers = 10)

test_dataset = PoseDataset(csv_file=os.path.join(ROOT_DIR,'test_joints.csv'),
                                  transform=transforms.Compose([
                                               Rescale((227,227)),
                                               ToTensor()
                                           ]))

test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers = 10)


In [17]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        model = models.resnet18(pretrained=True)
        for param in model.parameters():
            param.requires_grad = True
        model.fc=nn.Linear(512,32)
        self.resnet = model.cuda()
        
    def forward(self, x):
       
        pose_out = self.resnet(x)
        return pose_out

# OLD_VERSION = 8
PATH_PREFIX = '/home/yuliang/code/DeepPose-pytorch-yh/checkpoint/'
net_path = PATH_PREFIX + 'checkpoint{}.t7'.format(140)
# net = torch.load(net_path).cuda(device_id=gpus[0])
net = Net()
net = torch.nn.DataParallel(net).cuda()

# net = Net()
criterion = nn.MSELoss().cuda(device_id=gpus[0])
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=0.001, momentum=0.9)
optimizer = optim.SGD(net.parameters(), lr=0.0005, momentum=0.9)

RuntimeError: cuda runtime error (2) : out of memory at /pytorch/torch/lib/THC/generic/THCStorage.cu:66

In [18]:
import imgaug as ia
from imgaug import augmenters as iaa
from scipy import misc
import copy
import random
import numpy as np
import matplotlib.pyplot as plt
from imgaug import parameters as iap
plt.figure(figsize=(20,140))

def pose2keypoints(images, poses):
    keypoints_on_images = []
    for img_idx, (image, pose) in enumerate(zip(images, poses)):
        height, width = image.shape[0:2]
        keypoints = []
        for i in range(int(poses.shape[1] / 2)):
            x = int(poses[img_idx][i * 2])
            y = int(poses[img_idx][i * 2 + 1])
            keypoints.append(ia.Keypoint(x=x, y=y))
        keypoints_on_images.append(ia.KeypointsOnImage(keypoints, shape=image.shape))
    return keypoints_on_images

def keypoints2pose(keypoints_aug):
    all_keypoints = []
    for keypoints_after in keypoints_aug:
        one_person = []
        for kp_idx, keypoint in enumerate(keypoints_after.keypoints):
            x_new, y_new = keypoint.x, keypoint.y
            one_person.append(np.array(x_new).astype(np.float32))
            one_person.append(np.array(y_new).astype(np.float32))
        all_keypoints.append(np.array(one_person))
    return np.array(all_keypoints)    

def img_augmentation(images, poses, seq_dets):
    """
    images, poses :tensor;
    seq_dets: [iaa.Sequential]
    Example:
    seqs = [iaa.Sequential([iaa.Affine(rotate=45)]), iaa.Sequential([iaa.Fliplr(1.0)])]
    seq_dets = [seq.to_deterministic() for seq in seqs] 
    result_images, result_poses = img_augmentation(images, poses, seq_dets)
    """
    mean=np.array([0.485, 0.456, 0.406])
    std=np.array([0.229, 0.224, 0.225])
    glocal_cnt = 0
    np_images = images.numpy()
    np_poses = poses.numpy()
    out_images = copy.deepcopy(np_images)
    out_poses = copy.deepcopy(np_poses)
    # recover 
#     for i in range(np_images.shape[0]):
#         np_images[i] = np_images[i] * std + mean
    
    np_images = np.transpose(np_images, (0, 2, 3, 1))
    keypoints_on_images = pose2keypoints(np_images, np_poses)
    seq = iaa.Sequential([iaa.Fliplr(1.0)])
    seq_det = seq.to_deterministic() 
    for seq_det in seq_dets:
#         print("!!!!!!!!!!", np_images.shape)
#         np_images = np_images * std + mean
        images_aug = seq_det.augment_images(np_images)
#         images_aug = (images_aug - mean) / std
        keypoints_aug = seq_det.augment_keypoints(keypoints_on_images)
        new_keypoints = keypoints2pose(keypoints_aug)
        
        images_aug = np.transpose(images_aug, (0, 3, 1, 2)).astype(np.float32)
#         plt.clf()
#         for i in range(8):
#             display_pose2(images_aug[i], new_keypoints[i], glocal_cnt) 
#             glocal_cnt += 1
#         print(new_keypoints)
#         images_aug = np.transpose(images_aug, (0, 2, 3, 1))

#         print(out_images.shape, np_images.shape)
        
#         print(np_images.shape)    
        out_images = np.vstack((out_images, images_aug))
        out_poses = np.vstack((out_poses, new_keypoints))
#         print("out_images: ", out_images.shape)
    # force to float
    # change to tensor
    images = torch.from_numpy(out_images)
    poses = torch.from_numpy(out_poses)
    return (images, poses)

# seqs = [iaa.Sequential([iaa.Affine(rotate=45)]), iaa.Sequential([iaa.Fliplr(1.0)])]
# # call this for each batch again, NOT only once at the start
# seq_dets = [seq.to_deterministic() for seq in seqs] 
# result_images, result_poses = img_augmentation(images, poses, seq_dets)
# print(result_images.shape, result_poses.shape)

In [19]:
import pickle
import json
from tqdm import tqdm_notebook
import imgaug as ia
from imgaug import augmenters as iaa
from scipy import misc
import copy
import random
import numpy as np
import matplotlib.pyplot as plt
from imgaug import parameters as iap
import time
import numpy
# import gzip

# plt.figure(figsize=(20,140))

# aug_data = []
# max_aug_data_idx = 0

SAVE_PATH = '/disk3/yinghong/data/mobile-aug-data/'

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.integer):
            return int(obj)
        elif isinstance(obj, numpy.floating):
            return float(obj)
        elif isinstance(obj, numpy.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)
        
for i, data in tqdm_notebook(enumerate(train_dataloader)):
#     print("go!!!")
    if i % 100 == 0:
        print("i: %d" % i)
    images, poses = data['image'], data['pose']
    
    max_aug_data_idx = i
    seqs = []
    time1 = time.time()
#     for rotate in range(-180, 180, 45):
#         seqs.append(iaa.Sequential([iaa.Affine(rotate=rotate)]))
#     for scale_num in range(3, 8, 2):
#         seqs.append(iaa.Sequential([iaa.Affine(scale=(scale_num / 10.0 , scale_num / 10.0 + 0.1))]))

    seqs.append(iaa.Sequential([iaa.Fliplr(0.5)]))
#     seqs.append(iaa.Sequential([iaa.Flipud(1.0)]))

#     seqs.append(iaa.GaussianBlur(sigma=(0, 2)))
#     seqs.append(iaa.GaussianBlur(sigma=(0, 4)))
#     seqs.append(iaa.GaussianBlur(sigma=(0, 6)))
    seq_dets = [seq.to_deterministic() for seq in seqs] 
# #     print("pre images,", images[0])
    images, poses = img_augmentation(images, poses, seq_dets)
    

#     print("post images,", images[0])
    time2 = time.time()
    obj = {'images': images, 'poses': poses, 'idx': i, 'batch_size': BATCH_SIZE }
    time3 = time.time()
    torch.save(obj, SAVE_PATH + "aug-data-same-scale-0.5lr-type-bz%d-idx%d" % (BATCH_SIZE, i))
#     print("transform time=%d, save time=%d" % (time2 - time1, time3 - time2))


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

i: 0
i: 100
i: 200



In [ ]:
from tqdm import tqdm_notebook
import time
import matplotlib.pyplot as plt

import pickle
plt.figure(figsize=(20,140))

SAVE_PATH = "/disk3/yinghong/data/mobile-aug-data/"
def mse_loss(input, target):
    return torch.sum(torch.pow(input - target,2)) / input.nelement()

train_loss_all = []
valid_loss_all = []
# TBD(@yinghong): delete it after use
def display_pose(img, pose, ids):
    print
    mean=np.array([0.485, 0.456, 0.406])
    std=np.array([0.229, 0.224, 0.225])
#     pose  = pose.data.cpu().numpy().reshape([-1,2])
#     pose  = pose.numpy().reshape([-1,2])
    pose  = pose.numpy().reshape([-1,2])
#     img = img.cpu().numpy().transpose(1,2,0)
#     img = img.numpy().transpose(1,2,0)
    img = img.numpy().transpose(1,2,0)
    colors = ['b', 'g', 'g', 'g', 'g', 'g', 'm', 'm', 'r', 'r', 'y', 'y', 'y', 'y','y','y']
    pairs = [[8,9],[11,12],[11,10],[2,1],[1,0],[13,14],[14,15],[3,4],[4,5],[8,7],[7,6],[6,2],[6,3],[8,12],[8,13]]
    colors_skeleton = ['r', 'y', 'y', 'g', 'g', 'y', 'y', 'g', 'g', 'm', 'm', 'g', 'g', 'y','y']

    img = img*std+mean
#     print(img.shape) # (227, 227, 3)
    img_width, img_height,_ = img.shape
    plt.subplot(25,4,ids+1)
    ax = plt.gca()
    plt.imshow(img)
    for idx in range(len(colors)):
        plt.plot(pose[idx,0], pose[idx,1], marker='o', color=colors[idx])
    for idx in range(len(colors_skeleton)):
        plt.plot(pose[pairs[idx],0], pose[pairs[idx],1],color=colors_skeleton[idx])
    xmin = np.min(pose[:,0])
    ymin = np.min(pose[:,1])
    xmax = np.max(pose[:,0])
    ymax = np.max(pose[:,1])
    bndbox = np.array(expand_bbox(xmin, xmax, ymin, ymax, img_width, img_height))
    coords = (bndbox[0], bndbox[1]), bndbox[2]-bndbox[0]+1, bndbox[3]-bndbox[1]+1
    ax.add_patch(plt.Rectangle(*coords, fill=False, edgecolor='yellow', linewidth=1))
output_dict = {}

MAX_BATCH_SIZE = 561
IMG_WIDTH=227
IMG_HEIGHT=227
for epoch in tqdm_notebook(range(1000)):  # loop over the dataset multiple times
    train_loss_epoch = []
    beg_time = time.time()
#     for i, data in enumerate(train_dataloader):
    for i in tqdm_notebook(range(MAX_BATCH_SIZE)):
        if i % 100 == 0:
            print("current=%d" % i)
        # get the inputs
       
#         images, poses = data['image'], data['pose']
        obj = torch.load(SAVE_PATH + "aug-data-same-scale-bz32-idx%d" % i)
        images = obj['images']
#         print("images.size,", images[0])
        poses = obj['poses']
#         print("before:", images.shape, poses.shape)
#         print("before:", type(images), type(poses))
   
#         seqs = [iaa.Crop(px=(0, 16))]
        
###########data aug#############
#         seqs = []
#         for rotate in range(-180, 180, 45):
#             seqs.append(iaa.Sequential([iaa.Affine(rotate=rotate)]))
#         for scale_num in range(3, 8, 2):
#             seqs.append(iaa.Sequential([iaa.Affine(scale=(scale_num / 10.0 , scale_num / 10.0 + 0.1))]))
#         seqs.append(iaa.Sequential([iaa.Fliplr(1.0)]))
# #         seqs.append(iaa.Sequential([iaa.Flipud(1.0)]))
# #         seqs.append(iaa.GaussianBlur(sigma=(0, 1.0)))
#         seqs.append(iaa.GaussianBlur(sigma=(0, 2.0)))
# #         seqs.append(iaa.GaussianBlur(sigma=(0, 3.0)))

#         seq_dets = [seq.to_deterministic() for seq in seqs] 
#         images, poses = img_augmentation(images, poses, seq_dets)
#         print("after:", images.shape, poses.shape)
#         for i in range(4):
#             print("begin drawing")
#             plt.figure(figsize=(20,140))
#             display_pose(images[i], poses[i], i)
#             plt.show()
# #             print(poses[i])
            
#             plt.figure(figsize=(20,140))
#             display_pose(images[24 + i], poses[24 + i], i)
#             plt.show()
#             print(poses[24 + i])
            
#             plt.figure(figsize=(20,140))
#             display_pose(images[64 + i], poses[64 + i], i)
#             plt.show()
#             print(poses[128 + i])
#         print("before:", type(images), type(poses))

#         if i == 0:
# #             np.save(outfile, [images, poses])
#             pickle.dump(output_dict, open("output_dict.txt", "wb"))
#             print("dump done")

        # wrap them in Variable
        
        images, poses = Variable(images.cuda()), Variable(poses.cuda())
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(images)
        # TBD(@yinghong): change to a function
#         print(type(outputs[0][0]), type(poses[0][0]))
#         print(outputs[0][0].data[0])

#         for pose_idx in range(poses.size()[0]):
#             x = poses[pose_idx][0].data[0]
#             y = poses[pose_idx][1].data[0]
#             if x < 0 or x < 0 or y >= IMG_WIDTH or y >= IMG_HEIGHT:
#                 poses[pose_idx][0].data[0] = outputs[pose_idx][0].data[0]
#                 poses[pose_idx][1].data[0] = outputs[pose_idx][1].data[0]
        loss = criterion(outputs, poses)
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss_epoch.append(loss.data[0])
    print("epoch %d", epoch)
    end_time = time.time()
    if epoch%3==0:
        PATH_PREFIX = './'
        checkpoint_file = PATH_PREFIX + 'aug-data-same-scale-bz32-checkpoint{}.t7'.format(1 + epoch)
        torch.save(net, checkpoint_file)
        valid_loss_epoch = []
        for i_batch, sample_batched in enumerate(test_dataloader):

            net_forward = torch.load(checkpoint_file).cuda(device_id=gpus[0])
            images = sample_batched['image'].cuda(device=gpus[0])
            poses = sample_batched['pose'].cuda(device=gpus[0])
            outputs = net_forward(Variable(images, volatile=True))
            valid_loss_epoch.append(mse_loss(outputs.data,poses))
        print('[epoch %d] train loss: %.8f, valid loss: %.8f' %
          (epoch + 1, sum(train_loss_epoch)/(71*256), sum(valid_loss_epoch)/(8*256)))
        print('==> checkpoint model saving to %s'%checkpoint_file)
        train_loss_all.append(sum(train_loss_epoch)/(71*256))
        valid_loss_all.append(sum(valid_loss_epoch)/(8*256))
    print("Time:%d", end_time - beg_time)
            

print('Finished Training')


In [ ]:
# output_dict = pickle.load(open("output_dict.txt", "rb"))


In [ ]:
# print(output_dict['images'].shape)
# print(output_dict['poses'].shape)

In [ ]:
# import pickle
# output_dict = pickle.load(open("output_dict.txt", "rb"))
# print(output_dict['images'].shape)
# print(output_dict['poses'].shape)

In [ ]:
# print(output_dict['poses'].numpy()[:1])

In [ ]:
# import torch

# x1 = torch.zeros(10, 10)
# x2 = x1.unsqueeze(0)
# print(x2.size())
# sz=torch.Size([1, 10, 10])
# print(sz[0])

In [ ]:
import numpy
import torch
print (torch.__version__)

In [ ]:
import sys
print (sys.version)